In [1]:
import pandas as pd
import ast

# Load the dataset
df = pd.read_csv('rel.csv')

# Parse the JSON-like strings in the 'Trade_History' column
def parse_trade_history(trade_history_str):
    try:
        return ast.literal_eval(trade_history_str)
    except ValueError:
        return None

df['Trade_History'] = df['Trade_History'].apply(parse_trade_history)

# Drop rows with invalid 'Trade_History'
df = df.dropna(subset=['Trade_History'])

# Inspect the dataset
print(df.head())
print(df.info())
print(df.isnull().sum())


              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
<class 'pandas.core.frame.DataFrame'>
Index: 149 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       149 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.5+ KB
None
Port_IDs         0
Trade_History    0
dtype: int64


In [2]:
import numpy as np

# Define functions to calculate metrics
def calculate_roi(trade_history):
    if not trade_history:
        return 0
    initial_value = trade_history[0]['price'] * trade_history[0]['qty']
    final_value = trade_history[-1]['price'] * trade_history[-1]['qty']
    return (final_value - initial_value) / initial_value

def calculate_pnl(trade_history):
    return sum([trade['realizedProfit'] for trade in trade_history])

def calculate_sharpe_ratio(trade_history, risk_free_rate=0.01):
    returns = np.array([trade['realizedProfit'] for trade in trade_history])
    avg_return = np.mean(returns)
    std_return = np.std(returns)
    if std_return == 0:
        return np.nan  # Return NaN if the standard deviation is zero
    return (avg_return - risk_free_rate) / std_return

def calculate_mdd(trade_history):
    if not trade_history:
        return 0
    returns = np.array([trade['realizedProfit'] for trade in trade_history])
    cumulative_returns = np.cumsum(returns)
    peak = np.maximum.accumulate(cumulative_returns)
    drawdown = np.where(peak != 0, (cumulative_returns - peak) / peak, 0)  # Handle division by zero
    return drawdown.min() if len(drawdown) > 0 else 0

def calculate_win_rate(trade_history):
    win_positions = sum(1 for trade in trade_history if trade['realizedProfit'] > 0)
    total_positions = len(trade_history)
    return win_positions / total_positions if total_positions > 0 else 0

def calculate_win_positions(trade_history):
    return sum(1 for trade in trade_history if trade['realizedProfit'] > 0)

def calculate_total_positions(trade_history):
    return len(trade_history)


In [3]:
# Initialize a dictionary to store calculated metrics for each account
account_metrics = {}

# Group by account and calculate metrics
for port_id, group in df.groupby('Port_IDs'):
    trade_history = group['Trade_History'].values[0]
    metrics = {}
    metrics['ROI'] = calculate_roi(trade_history)
    metrics['PnL'] = calculate_pnl(trade_history)
    metrics['Sharpe_Ratio'] = calculate_sharpe_ratio(trade_history)
    metrics['MDD'] = calculate_mdd(trade_history)
    metrics['Win_Rate'] = calculate_win_rate(trade_history)
    metrics['Win_Positions'] = calculate_win_positions(trade_history)
    metrics['Total_Positions'] = calculate_total_positions(trade_history)
    account_metrics[port_id] = metrics

# Convert the dictionary to a DataFrame
metrics_df = pd.DataFrame.from_dict(account_metrics, orient='index')

# Replace any NaN values with 0 (optional)
metrics_df.fillna(0, inplace=True)

# Inspect the calculated metrics
print(metrics_df.head())


C:\Users\pjee1\AppData\Local\Temp\ipykernel_25768\2568363956.py:28: RuntimeWarning: invalid value encountered in divide
  drawdown = np.where(peak != 0, (cumulative_returns - peak) / peak, 0)  # Handle division by zero
C:\Users\pjee1\AppData\Local\Temp\ipykernel_25768\2568363956.py:28: RuntimeWarning: divide by zero encountered in divide
  drawdown = np.where(peak != 0, (cumulative_returns - peak) / peak, 0)  # Handle division by zero


                          ROI          PnL  Sharpe_Ratio       MDD  Win_Rate  \
3672754654734989568  0.129044   566.597660      0.183919 -0.460781  0.443038   
3733192481840423936 -0.781360  2923.977200      0.060167 -0.760355  0.802612   
3768170840939476993 -0.989951   243.668899      0.440041  0.000000  0.428571   
3784403294629753856 -0.742824  2521.814305      0.104037 -0.177261  0.302314   
3786761687746711808 -0.791718   205.021400      0.215609 -2.318212  0.451220   

                     Win_Positions  Total_Positions  
3672754654734989568            210              474  
3733192481840423936            553              689  
3768170840939476993              6               14  
3784403294629753856           1829             6050  
3786761687746711808             37               82  


In [4]:
# Define weights for each metric (example weights)
weights = {
    'ROI': 0.3,
    'PnL': 0.2,
    'Sharpe_Ratio': 0.2,
    'MDD': 0.1,
    'Win_Rate': 0.1,
    'Win_Positions': 0.05,
    'Total_Positions': 0.05
}

# Calculate the final score for each account
metrics_df['Final_Score'] = (metrics_df['ROI'] * weights['ROI'] +
                             metrics_df['PnL'] * weights['PnL'] +
                             metrics_df['Sharpe_Ratio'] * weights['Sharpe_Ratio'] -
                             metrics_df['MDD'] * weights['MDD'] +
                             metrics_df['Win_Rate'] * weights['Win_Rate'] +
                             metrics_df['Win_Positions'] * weights['Win_Positions'] +
                             metrics_df['Total_Positions'] * weights['Total_Positions'])

# Rank the accounts based on the final score
metrics_df['Rank'] = metrics_df['Final_Score'].rank(ascending=False)

# Save the calculated metrics to a CSV file
metrics_df.to_csv('calculated_metrics.csv', index=True)

# Get the top 20 accounts based on rank
top_20_accounts = metrics_df.sort_values(by='Rank').head(20)

# Save the top 20 accounts to a CSV file
top_20_accounts.to_csv('top_20_accounts.csv', index=True)

# Display the top 20 accounts
print(top_20_accounts)


                          ROI           PnL  Sharpe_Ratio        MDD  \
4020204877254599680 -0.999880  71998.855953      0.060702  -8.028348   
3999240873283311617 -0.990422  42574.473679      0.227289  -0.052816   
4021669203289716224 -0.953461  26427.331592      0.068228  -0.161568   
3960874214179953664 -0.900346  19567.471286      0.162783 -34.536210   
3907081197088384000 -0.140110  18015.997370      0.222245  -0.242908   
3956076827719377409  0.135495  16790.012238      0.099309  -0.246143   
3986814617275053313 -0.482620  16337.461881      0.329115  -0.000149   
4028701921959171840  0.729821  17601.401398      0.275817  -0.045177   
3788465932399412480 -0.701269  13960.966457      0.152601  -0.224263   
4022565861939831809  1.221727  14197.577076      0.299666  -0.108960   
3987739404272887297 -0.816087  12464.606315      0.049644  -0.310502   
3931992636670880512 -0.708627  10374.965290      0.050504  -0.424280   
4008711265867865600 -0.964153  11298.706390      0.266535  -0.05